<a href="https://colab.research.google.com/github/RonildoSilva/datasets/blob/main/big_black_forecast_da_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from datetime import datetime

! pip install flood-forecast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
! pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import wget
import os.path

In [28]:
#! unzip 01010500FVE_flow.csv.zip

In [29]:
import wget
import os.path

if(not os.path.exists('df_train_keras_tk.csv')):
    wget.download('https://raw.githubusercontent.com/RonildoSilva/datasets/main/df_train_keras_tk.csv')

if(not os.path.exists('df_val_keras_tk.csv')):
    wget.download('https://raw.githubusercontent.com/RonildoSilva/datasets/main/df_val_keras_tk.csv')

if(not os.path.exists('df_test_keras_tk.csv')):
    wget.download('https://raw.githubusercontent.com/RonildoSilva/datasets/main/df_test_keras_tk.csv')

Merge

In [30]:
import pandas as pd 
df = pd.concat([
    pd.read_csv('df_train_keras_tk.csv'),
    pd.read_csv('df_val_keras_tk.csv'),
    pd.read_csv('df_test_keras_tk.csv')
])

In [31]:
print(
    pd.read_csv('df_train_keras_tk.csv').shape,
    pd.read_csv('df_val_keras_tk.csv').shape,
    pd.read_csv('df_test_keras_tk.csv').shape
)

(12363, 20) (4021, 20) (4020, 20)


In [62]:
DATETIME_START = pd.to_datetime(pd.read_csv('df_test_keras_tk.csv')['datetime']).dt.strftime('%Y-%m-%d').min()

In [63]:
DATETIME_END = pd.to_datetime(pd.read_csv('df_test_keras_tk.csv')['datetime']).dt.strftime('%Y-%m-%d').max()

In [64]:
DATETIME_START, DATETIME_END

('2010-01-13', '2014-01-03')

In [65]:
TRAIN_START = 0
TRAIN_END = pd.read_csv('df_train_keras_tk.csv').shape[0]

VAL_START = TRAIN_END + 1
VAL_END = VAL_START + pd.read_csv('df_val_keras_tk.csv').shape[0]

TEST_START = VAL_END + 1
TEST_END = TEST_START + pd.read_csv('df_test_keras_tk.csv').shape[0] - 1

In [66]:
print(TRAIN_START, TRAIN_END, VAL_START, VAL_END, TEST_START, TEST_END)

0 12363 12364 16385 16386 20405


In [36]:
df['datetime'] = pd.to_datetime(df['datetime']).dt.strftime('%Y-%m-%d')
#df['datetime'] = pd.to_datetime(df['datetime']).dt.strftime('%Y-%m-%d %H:%M:%S+00:00')
#2000-04-04 04:00:00+00:00

In [37]:
#df = df[["TimeConclusion", "Duration", "datetime"]]
df.to_csv('df_full.csv', index=False)

In [38]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,datetime,Duration,PassedTime,Step,TimeConclusion
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2012-10-09,0.0000,0.0000,1,31.0087
1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2012-10-09,16.0084,15.0002,2,15.0003
2,1,2,3,0,0,0,0,0,0,0,0,0,0,0,0,2012-10-12,0.0001,15.0003,3,15.0002
3,1,2,3,5,0,0,0,0,0,0,0,0,0,0,0,2012-10-25,15.0002,31.0087,4,0.0000
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2012-11-09,0.0000,0.0000,1,30.9822


In [39]:
"""
!pip install -Uqq ipdb
import ipdb
%pdb on
"""

'\n!pip install -Uqq ipdb\nimport ipdb\n%pdb on\n'

In [40]:
FEATURE_COLS = df.columns.tolist()
FEATURE_COLS.remove('datetime')

TARGET_COL = ["TimeConclusion"]

In [41]:
DIMENSION_FEAT = len(FEATURE_COLS)

In [42]:
DIMENSION_FEAT

19

In [43]:
print(FEATURE_COLS, sep=' ')

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', 'Duration', 'PassedTime', 'Step', 'TimeConclusion']


In [44]:
print(TARGET_COL)

['TimeConclusion']


Config

In [67]:
def make_config_file(flow_file_path, gage_id, station_id, weight_path=None, pretrained=[]):
    run = wandb.init(project="github_aistream-peelout_flow-forecast")
    wandb_config = run.config
    the_wandb_c = run.config
    print(wandb_config)
    the_config4 = {"model_name": "DARNN",
        "model_type": "PyTorch",
        "model_params": {
        "n_time_series": (2+DIMENSION_FEAT),
        "forecast_history": 14,
        "hidden_size_encoder":32,
        "decoder_hidden_size": 32,
        "out_feats": 1,
        "dropout": 0.1,
        "gru_lstm": False},
       "dataset_params":{"class": "default",
       "num_workers":2,
       "pin_memory": True,
       "training_path": flow_file_path,
       "validation_path": flow_file_path,
       "test_path": flow_file_path,
       "batch_size":200,
       "forecast_history": (14 - 1),
       "forecast_length": 1,
       "scaler": "StandardScaler",
       "train_start": TRAIN_START,
       "train_end": TRAIN_END,
       "valid_start": VAL_START,
       "valid_end": VAL_END,
       "sort_column": "datetime",# "hour_updated",
       "test_start": TEST_START,
       "test_end": TEST_END,
       "target_col": TARGET_COL,
       "relevant_cols": FEATURE_COLS, 
       "interpolate":{
           "method":"back_forward_generic",
           "params":{
               "relevant_columns":FEATURE_COLS}
           },
      "feature_param":
          {
            "datetime_params":{
            "day":"numerical",
            "month": "numerical"
            }
          }
       },
    "training_params":
    {
       "criterion":"L1",
       "optimizer": "SGD",
       #"criterion_params":{"baseline_method":"mean"},
    "optim_params":{
       "lr": 0.001
    },
       "epochs": 12,
       "batch_size": 200
    },
    "early_stopping":{
        "patience":3
    },
    "GCS": False,
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE", "L1"],
   "inference_params":
   {     
          "datetime_start":DATETIME_START,
          "hours_to_forecast": 72, 
          "num_prediction_samples": 20,
          "test_csv_path":flow_file_path,
          "decoder_params":{
            "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1},
          "dataset_params":{
             "file_path": flow_file_path,
             "sort_column": "datetime",
             "scaling": "StandardScaler",
             "forecast_history": (14 - 1),
             "forecast_length":1,
             "relevant_cols": FEATURE_COLS,
             "target_col": TARGET_COL,
             "interpolate_param":{
                 "method":"back_forward_generic",
                 "params":{"relevant_columns":FEATURE_COLS}
                 }, 
            "feature_params":
        {
         "datetime_params":{
            "day":"numerical",
            "month": "numerical"
         
     }
             }
          }
          } 
    }

      
    if weight_path:
        the_config4["weight_path"] = weight_path
    wandb.config.update(the_config4)
    print("config made")
    return the_config4
  
wandb_sweep_config_full = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "forecast_length":{
            "values":[1]}
            ,
        "batch_size": {
            "values": [200]#, 400, 600]
        },
        "lr":{
            "values":[0.001]#, 0.0001, .01]
        },
        "forecast_history":{
            "values":[14]#, 20, 24, 48]
        },
        "out_seq_length":{
            "values":[1]
        },
        "hidden_encoder":
        {
            "values":[32]#, 64, 128, 256]
        },
        "hidden_decoder":
        {
            # 32, 64, 128 
            "values":[32]#, 64, 128, 256]
        }, 
        "dropout":
        {
            "values": [0.1],#, 0.3, 0.5, 0.8]
        }
  
    }
}

In [68]:
import wandb
from flood_forecast.trainer import train_function


sweep_id = wandb.sweep(wandb_sweep_config_full, project="github_aistream-peelout_flow-forecast")
#sweep_id = "03l75ilz"

#file_path = "01010500FVE_flow.csv"
file_path = "df_full.csv"

#wandb.agent(sweep_id, lambda: train_function("PyTorch", make_config_file(file_path, "01010500", "FVE", None)))

model = train_function("PyTorch", make_config_file(file_path, "01010500", "FVE", None))

Create sweep with ID: 1s171u05
Sweep URL: https://wandb.ai/ronildosilva/github_aistream-peelout_flow-forecast/sweeps/1s171u05


average_prediction_sharpe0,▁
epoch,▁▁▂▂▄▄▅▅▇▇███
loss,█▄▂▂▁▁
average_prediction_sharpe0,0.25691
epoch,5
loss,0.22225


{}
config made
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['day', 'month']
scaling now
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['day', 'month']
scaling now
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['day', 'month']
scaling now
Using Wandb config:
{'model_name': 'DARNN', 'model_type': 'PyTorch', 'model_params': {'n_time_series': 21, 'forecast_history': 14, 'hidden_size_encoder': 32, 'decoder_hidden_size': 32, 'out_feats': 1, 'dropout': 0.1, 'gru_lstm': False}, 'dataset_params': {'class': 'default', 'num_workers': 2, 'pin_memory': True, 'training_path': 'df_full.csv', 'validation_path': 'df_full.csv', 'test_path': 'df_full.csv', 'batch_size': 200, 'forecast_history': 13, 'forecast_length': 1, 'scaler': 'StandardScaler', 'train_start': 0, 'train_end': 12363, 'valid_start': 12364, 'valid_end': 16385, 's

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1053: UserWarning:

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1043: UserWarning:

Using a non-full backward hook when outputs are generated by different autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_output. Please use register_full_backward_hook to get the documented behavior.



plotting with CI now
Now plotting final plots


In [69]:
model.test_data.df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,Duration,PassedTime,Step,TimeConclusion,day,month
6156,-0.197868,0.411135,0.593365,2.008866,-0.357816,-0.20214,-0.114685,-0.09391,-0.061848,-0.047687,...,-0.026473,-0.015778,0.0,0.0,1.596024,0.531093,0.728307,-1.518155,5,9
16218,-0.197868,0.411135,1.139684,0.946181,3.388155,-0.20214,-0.114685,-0.09391,-0.061848,-0.047687,...,-0.026473,-0.015778,0.0,0.0,0.410794,0.359955,1.377738,-1.518155,5,9
6178,-0.197868,0.411135,0.593365,2.008866,-0.357816,-0.20214,-0.114685,-0.09391,-0.061848,-0.047687,...,-0.026473,-0.015778,0.0,0.0,1.742839,0.531210,0.728307,-1.518155,5,9
15586,0.914066,2.487743,0.593365,-0.647846,-0.357816,-0.20214,-0.114685,-0.09391,-0.061848,-0.047687,...,-0.026473,-0.015778,0.0,0.0,1.125132,0.701112,0.078876,-0.682865,5,9
19618,-0.197868,0.411135,-1.045594,-0.647846,-0.357816,-0.20214,-0.114685,-0.09391,-0.061848,-0.047687,...,-0.026473,-0.015778,0.0,0.0,0.607161,-0.691494,-0.570555,-0.627119,5,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5364,5.361801,-1.665472,-1.045594,-0.647846,-0.357816,-0.20214,-0.114685,-0.09391,-0.061848,-0.047687,...,-0.026473,-0.015778,0.0,0.0,-0.658879,-1.526494,-1.219986,1.821177,3,1
19218,-0.197868,0.411135,0.593365,-0.647846,-0.357816,-0.20214,-0.114685,-0.09391,-0.061848,-0.047687,...,-0.026473,-0.015778,0.0,0.0,-0.428211,-0.511439,0.078876,-0.682865,3,1
13239,-0.197868,0.411135,-1.045594,-0.647846,-0.357816,-0.20214,-0.114685,-0.09391,-0.061848,-0.047687,...,-0.026473,-0.015778,0.0,0.0,-0.658864,0.152918,-0.570555,0.208188,3,1
1489,5.361801,-1.665472,-1.045594,-0.647846,-0.357816,-0.20214,-0.114685,-0.09391,-0.061848,-0.047687,...,-0.026473,-0.015778,0.0,0.0,-0.658879,-1.526494,-1.219986,0.765040,3,1
